# Check data sources

The first thing I want to do is check what solar edge systems have metering or battery information. Right now I know of two that have metering and one that also has battery. I would like to know if there are any more

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
from libraries.solar_edge_api import SolarEdgeApi
se_api = SolarEdgeApi(True)

/Users/richardtimpson/Documents/energbase/SchoolProject/code/libraries


In [4]:
sites = se_api.get_site_list()

Making SolarEdge Api call
Request succeeded!


In [5]:
site_md = sites['sites']['site']

In [6]:
for site in site_md:
    site_id = site['id']
    inventory = se_api.get_site_inventory(site_id)

    if inventory:
        meters = False
        batteries = False

        if inventory['Inventory']['meters']:
            meters = True

        if inventory['Inventory']['batteries']:
            batteries = True

        if meters or batteries:
            print(f'site {site["name"]} ({site_id}) has {"meters" if meters else ""} {"batteries" if batteries else ""}')

Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
site Huntington Beach (896164) has meters batteries
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
site Lot 5 CopperRock (1520780) has meters 
Making SolarEdge Api call
Request succeeded!
site Lot 8 Copper Rock (1520756) has meters 
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!
site Ronald L. McClellan Sr. (466851) has meters 
Making SolarEdge Api call
Request succeeded!
Making SolarEdge Api call
Request succeeded!


My initial assumptions were verified. We want to get consumption data for system 896164, 1520780, 1520756, and 466851

# Gather SolarEdge Consumption data

In [11]:
import datetime

site_ids = [896164, 1520780, 1520756, 466851]
consumption_data = {site_id: None for site_id in site_ids }
for site in site_md:
    if site['id'] in site_ids:
        data_period = se_api.get_data_period(site['id'])['dataPeriod']
        print(data_period)
        time_fmt = '%Y-%m-%d'
        start = datetime.datetime.strptime(data_period['startDate'], time_fmt)
        end = datetime.datetime.strptime(data_period['endDate'], time_fmt) 
        data = se_api.get_site_consumption_half_hour_range(site['id'], start, end)
        consumption_data[site['id']] = data 

Making SolarEdge Api call
Request succeeded!
{'startDate': '2018-11-06', 'endDate': '2020-04-07'}
2018-11-06 00:00:00 2018-12-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2018-12-06 00:00:00 2019-01-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-01-06 00:00:00 2019-02-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-02-06 00:00:00 2019-03-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-03-06 00:00:00 2019-04-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-04-06 00:00:00 2019-05-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-05-06 00:00:00 2019-06-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-06-06 00:00:00 2019-07-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-07-06 00:00:00 2019-08-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-08-06 00:00:00 2019-09-06 00:00:00
Making SolarEdge Api call
Request succeeded!
2019-09-06 00:00:00 2019-10-06 00:00:00
Making Solar

In [18]:
import pandas as pd


consumption_data_dfs = {}
for key, value in consumption_data.items():
    dfs = []
    # print(value)
    for d in value:
        data = None 
        for x in d['energyDetails']['meters']:
            if x['type'] == 'Consumption':
                data = x['values']
        df = pd.DataFrame(data)
        dfs.append(df)
    df = pd.concat(dfs)
    consumption_data_dfs[key] = df

In [19]:
consumption_data_dfs[896164]

,date,value
0,2018-11-06 00:00:00,NaN
1,2018-11-06 00:15:00,NaN
2,2018-11-06 00:30:00,NaN
3,2018-11-06 00:45:00,NaN
4,2018-11-06 01:00:00,NaN
...,...,...
2971,2020-04-05 22:45:00,313.0
2972,2020-04-05 23:00:00,282.0
2973,2020-04-05 23:15:00,341.0
2974,2020-04-05 23:30:00,290.0
